In [1]:
import skopt
from skopt import gp_minimize, forest_minimize
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_convergence
from skopt.plots import plot_objective, plot_evaluations
#from skopt.plots import plot_histogram, plot_objective_2D
from skopt.utils import use_named_args

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import gc
import time
import os
import sys
#sys.path.append(os.path.join(os.path.dirname(__file__), '../'))
sys.path.append('../models')

from data_frame import DataFrame
from tf_base_model import TFBaseModel # for building our customized

config = tf.ConfigProto(allow_soft_placement = True)
config.gpu_options.allow_growth=True

class DataReader(object):
    '''for reading data'''
    
    def __init__(self, data_dir):
        data_cols = [
            'item_id',
            'word_id',
            'history_length',
            'label'
        ]
        #-----------------
        # loading data
        #-----------------
        if TRACE_CODE == True:
            data = [np.load(os.path.join(data_dir, '{}_0.npy'.format(i)), mmap_mode='r') for i in data_cols]
        else:
            data = [np.load(os.path.join(data_dir, '{}.npy'.format(i)), mmap_mode='r') for i in data_cols]
        #------------------
        # For Testing-phase
        #------------------
        self.test_df = DataFrame(columns=data_cols, data=data)

        print ('shape of whole data : {}'.format(self.test_df.shapes()))
        print ('loaded data')
        #------------------
        # For Training-phase
        #------------------

        self.train_df, self.val_df = self.test_df.train_test_split(train_size=0.9, random_state = int(time.time()))

        print ('number of training example: {}'.format(len(self.train_df)))
        print ('number of validating example: {}'.format(len(self.val_df)))
        print ('number of testing example: {}'.format(len(self.test_df)))
        
    def train_batch_generator(self, batch_size, shuffle = True):
        return self.batch_generator(
            batch_size=batch_size,
            df=self.train_df,
            shuffle=shuffle,
            num_epochs=10000,
            is_test=False
        )

    def val_batch_generator(self, batch_size, shuffle = True):
        return self.batch_generator(
            batch_size=batch_size,
            df=self.val_df,
            shuffle=shuffle,
            num_epochs=10000,
            is_test=False
        )

    def test_batch_generator(self, batch_size,shuffle = False):
        return self.batch_generator(
            batch_size=batch_size,
            df=self.test_df,
            shuffle=shuffle,
            num_epochs=1,
            is_test=True
        )
    def batch_generator(self, batch_size, df, shuffle=True, num_epochs=10000, is_test=False):
        '''
        df: customized DataFrame object,
        '''
        # call our customized DataFrame object method batch_generator
        batch_gen = df.batch_generator(batch_size, shuffle = shuffle, num_epochs=num_epochs, allow_smaller_final_batch=is_test)
        # batch_gen is a generator
        for batch in batch_gen:
            # what batch_gen yield is also a customized Dataframe object.
            if not is_test:
                pass
            yield batch


/home/linuxbrew/.linuxbrew/opt/python/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
TRACE_CODE = False
dr = DataReader(data_dir ='../models/data/')


shape of whole data : item_id               (24028,)
word_id           (24028, 122)
history_length        (24028,)
label             (24028, 122)
dtype: object
loaded data
number of training example: 21625
number of validating example: 2403
number of testing example: 24028


/home/linuxbrew/.linuxbrew/opt/python/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [4]:
from tf_utils import temporal_convolution_layer
from tf_utils import time_distributed_dense_layer
from tf_utils import sequence_softmax_loss
from data_utils import get_glove_vectors
from data_utils import load_vocab_and_return_word_to_id_dict


In [5]:
sys.path.append('../models/cnn') 
from simple_cnn import simple_cnn

In [6]:
# setting
TRACE_CODE = False
dim_word = 300
trainable_embedding = False
USE_PRETRAINED = True
filename_words_vec = "../models/data/wordvec/word2vec.npz".format(dim_word)
filename_words_voc = "../models/data/wordvec/words_vocab.txt"
base_dir = ''
# load pre-trained word embedding
nwords = len(load_vocab_and_return_word_to_id_dict(filename_words_voc))
embeddings = (get_glove_vectors(filename_words_vec) if USE_PRETRAINED else None)
embeddings = embeddings.astype(np.float32)
enable_parameter_averaging = False

nn = simple_cnn(
    reader=dr,
    log_dir=os.path.join(base_dir, 'logs'),
    checkpoint_dir=os.path.join(base_dir, 'checkpoints'),
    prediction_dir=os.path.join(base_dir, 'predictions'),
    optimizer='adam',
    learning_rate =0.001,
    hidden_size_cnn = 300,
    filter_widths= 3,
    num_hidden_layers =2,
    ntags = 3,
    batch_size = 128,
    dim_word = 300,
    nwords = nwords,
    trainable_embedding = False,
    embeddings = embeddings,
    num_training_steps=200000,
    early_stopping_steps=30000,
    warm_start_init_step = 0, # for some case, we don't want to train the model from the beginning
    regularization_constant = 0.0,
    keep_prob = 1.0,
    enable_parameter_averaging = False,
    num_restarts = 2,
    min_steps_to_checkpoint = 5000,
    log_interval = 20,
    num_validation_batches = 4,
)



new run with parameters:
{'batch_size': 128,
 'checkpoint_dir': 'checkpoints',
 'dim_word': 300,
 'early_stopping_steps': 30000,
 'embeddings': array([[-4.391240e-01,  4.646779e+00, -1.602939e+00, ..., -9.194760e-01,
        -8.168350e-01,  3.141200e-02],
       [-5.499886e+00,  1.631320e+00, -1.703313e+00, ...,  1.883183e+00,
         4.813218e+00, -2.075190e-01],
       [ 3.052127e+00,  1.373138e+00,  1.502869e+00, ..., -9.166140e-01,
        -1.385339e+00,  9.958170e-01],
       ...,
       [-2.782620e+00, -4.530440e-01,  3.562689e+00, ...,  3.935201e+00,
         9.740690e-01, -1.321913e+00],
       [ 1.867000e-03, -2.541924e+00, -4.685680e-01, ...,  1.460600e-02,
        -4.422500e-02,  2.060565e+00],
       [ 1.605634e+00,  2.821861e+00, -3.930379e+00, ...,  1.326763e+00,
         2.236029e+00,  2.455661e+00]], dtype=float32),
 'enable_parameter_averaging': False,
 'filter_widths': 3,
 'grad_clip': 5,
 'hidden_size_cnn': 300,
 'keep_prob_scalar': 1.0,
 'learning_rate': 0.001,
 '

step - whtat optimizer.apply_gradients returns name: "Adam"
op: "AssignAdd"
input: "Variable"
input: "Adam/value"
attr {
  key: "T"
  value {
    type: DT_INT32
  }
}
attr {
  key: "_class"
  value {
    list {
      s: "loc:@Variable"
    }
  }
}
attr {
  key: "use_locking"
  value {
    b: false
  }
}

built graph


# Step1:We first need to define the valid search-ranges or search-dimensions for each of these parameters.

In [ ]:
dim_learning_rate = Real(low=1e-6, high=1e-2, prior='log-uniform',
                         name='learning_rate')
# We then combine all these search-dimensions into a list.
search_space = [
    dim_learning_rate,
#     dim_num_dense_layers,
#     dim_num_dense_nodes,
#     dim_activation
]

# Step2: choosing starting point

In [ ]:
# It is helpful to start the search for hyper-parameters with a decent choice that we have found by hand-tuning. 
default_parameters = [1e-5, 1, 16, 'relu']
default_parameters

# Step3: Define the function that evaluates its performance on the validation-set. 

In [ ]:
def log_dir_name(learning_rate):
    '''log the training-progress for all parameter-combinations'''
    # The dir-name for the TensorBoard log-dir.
    s = "./logs/lr_{}_layers_{}_nodes_{}_{}/"

    # Insert all the hyper-parameters in the dir-name.
    log_dir = s.format(
                       learning_rate,
#                        num_dense_layers,
#                        num_dense_nodes,
#                        activation
    )

    return log_dir

# function decorator @use_named_args which wraps the fitness function so that it can be called with all the parameters as a single list, for example: fitness(x=[1e-4, 3, 256, 'relu']). 
# This is the calling-style skopt uses internally.
@use_named_args(search_space)
def fitness(learning_rate):
    """
    Hyper-parameters:
    learning_rate:     Learning-rate for the optimizer.
    num_dense_layers:  Number of dense layers.
    num_dense_nodes:   Number of nodes in each dense layer.
    activation:        Activation function for all layers.
    """

    # Print the hyper-parameters.
    print('learning rate: {}'.format(learning_rate))
#     print('num_dense_layers:', num_dense_layers)
#     print('num_dense_nodes:', num_dense_nodes)
#     print('activation:', activation)
    #-------------------------
    # Customized model: Create the neural network with these hyper-parameters.
    #-------------------------
    # trying to using tensorflow model
#     model = create_model(learning_rate=learning_rate,
#                          num_dense_layers=num_dense_layers,
#                          num_dense_nodes=num_dense_nodes,
#                          activation=activation)
    nn = simple_cnn(
        reader=dr,
        log_dir=os.path.join(base_dir, 'logs'),
        checkpoint_dir=os.path.join(base_dir, 'checkpoints'),
        prediction_dir=os.path.join(base_dir, 'predictions'),
        optimizer='adam',
        learning_rate = learning_rate,
        hidden_size_cnn = 300,
        filter_widths= 3,
        num_hidden_layers =2,
        ntags = 3,
        batch_size = 128,
        dim_word = 300,
        nwords = nwords,
        trainable_embedding = False,
        embeddings = embeddings,
        num_training_steps=200000,
        early_stopping_steps=30000,
        warm_start_init_step = 0, # for some case, we don't want to train the model from the beginning
        regularization_constant = 0.0,
        keep_prob = 1.0,
        enable_parameter_averaging = False,
        num_restarts = 2,
        min_steps_to_checkpoint = 5000,
        log_interval = 20,
        num_validation_batches = 4,
    )

#     # Dir-name for the TensorBoard log-files.
#     log_dir = log_dir_name(learning_rate, num_dense_layers,
#                            num_dense_nodes, activation)
    log_dir = log_dir_name(learning_rate)
    #-------------------------
    # model training 
    #-------------------------
#     history = model.fit(x=data.train.images,
#                         y=data.train.labels,
#                         epochs=3,
#                         batch_size=128,
#                         validation_data=validation_data,
#                         callbacks=[callback_log])
    nn.fit() # need a method return loss on the validation-set create evaluatio(val_set)
#     #-------------------------
#     # Get the loss(classification accuracy) on the validation-set
#     # loss = objective_fun()
#     #-------------------------
#     accuracy = history.history['val_acc'][-1]

#     # Print the classification accuracy.
#     print()
#     print("Accuracy: {0:.2%}".format(accuracy))
#     print()

#     # Save the model if it improves on the best-found performance.
#     # We use the global keyword so we update the variable outside
#     # of this function.
#     global best_accuracy # best_loss

#     # If the classification accuracy of the saved model is improved ...
#     if accuracy > best_accuracy:
#         # Save the new model to harddisk.
#         model.save(path_best_model)
        
#         # Update the classification accuracy.
#         best_accuracy = accuracy # best_loss

#     # Delete the Keras model with these hyper-parameters from memory.
#     del model
    
#     return -accuracy # loss
    return 

In [ ]:
fitness